# PyDAP as a client


`PyDAP` can be used to \"lazily\"  inspect and retrieve remote data from any of the thousands of scientific datasets available on the internet on [OPeNDAP](http://www.opendap.org/) data servers, allowing the user to manipulate a `Dataset` as if it were stored locally, only downloading on-the-fly when necessary. In order to transmit data from the Server to the Client, `both server and client must agree on a way to represent data`: **is it an array of integers?**, **a multi-dimensional grid?** In order to do this, a DAP protocol defines a **data model** that, in theory, should be able to represent any existing (scientific) dataset.


Pydap uses the `requests` library to fetch remote data from an OPeNDAP data server. Data from such a server is one of the following types:

| File Extension | File Type | Protocol | Example URL |
| :- | :- | :- | -: | 
| DMR | Metadata | DAP4 | http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dmr |
| DAP | Metadata and binary |  DAP4 | http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dap |
| DDS | Metadata | DAP2  | http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dds |
| DAS | Metadata | DAP2 | http://test.opendap.org/opendap/data/nc/coads_climatology.nc.das |
| DODS | Metadata and Binary | DAP2 | http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dods | 

```{note}
Clickling on any of the `dap` or `dods` example URLs will trigger a download of a OPeNDAP binary data. Pydap parses this binary data and turns is into a pydap Dataset.
```






## Requests library

As of version `3.5.4`, `pydap` now uses Python's requests library to get/fetch the remote datasets described on the Table above and can also use Python's `requests_cache` library to cache responses. For the user, pydap has a special function to initialize any such session:


| Session with No Cache | Cached Session |
| :- | :- |
| use_cache=False (default) | use_cache=True |


In [ ]:
from pydap.client import open_url
from pydap.net import create_session

In [ ]:
data_url = "http://test.opendap.org/opendap/data/nc/coads_climatology.nc"

## Use default non-cached Session


In [ ]:
# default
my_session = create_session()

In [ ]:
%%time
pyds = open_url(data_url, protocol="dap4", session=my_session)

### Lets try again

In [ ]:
%%time
pyds = open_url(data_url, protocol="dap4", session=my_session)

### What is hapenning?

In both cases, only the `dmr` associated with the remote dataset was fetched, and used to create the pydap dataset.

The apparent difference in timing can sometimes be attributes to what is called "cold reading" vs "warm reading". But in both scenarios, 
each time the `pyds` is created, the remote `dmr` dataset is fetched and processed by pydap to create the `lazy` dataset that point
to the original opendap source.

To avoid repeatedly downloading the same resource over and over, potentially overwhelming remote data servers, pydap can now cache responses. 


## Use Cached-Session

In [ ]:
# Non-default
cached_session = create_session(use_cache=True)

### clear any prevous cached session

In [ ]:
cached_session.cache.clear()

In [ ]:
%%time
new_pyds = open_url(data_url, protocol="dap4", session=cached_session)


The timing required to download a remote `dmr` from the same server remains close to that of the `warm` case.

### Now let's try again!

In [ ]:
%%time
new_pyds = open_url(data_url, protocol="dap4", session=cached_session)


The resulting timing has dropped significantly. This is because the `dmr` was never downloaded from the remote source. Insted it was fetched form the cache.

In [ ]:
print("Default location of cached response: ", cached_session.cache.db_path)

In [ ]:
print("URLs of cached responses: ", cached_session.cache.urls())

### Finally, let's clear the cache

In [ ]:
cached_session.cache.clear()

In [ ]:
print("URLs of cached responses: ", cached_session.cache.urls())


## Timeout

To specify a timeout for the client, just set the desired number of seconds using the `timeout` option to `open_url(...)`. For example, the following would timeout after 30 seconds without receiving a response from the server:

```{code}
dataset = open_url('http://test.opendap.org/dap/data/nc/coads_climatology.nc', timeout=30)
```

```{note}
The default timeout is 120 seconds, or 2 minutes.
```